In [11]:
from sklearn.metrics import mean_squared_error
import pandas as pd

import mlflow
from mlflow.tracking import MlflowClient

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [12]:
link = 'https://d37ci6vzurychx.cloudfront.net/trip-data/'
data = 'green_tripdata_2021-03.parquet'

datapath = link + data
df = read_dataframe(datapath)

In [4]:
MLFLOW_TRACKING_URI = 'sqlite:///mlflow.db'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [18]:
run_id = '6c04258530374828bacfc41366c414cc'
model_uri = f'runs:/{run_id}/model'
model_name = 'nyc-taxi-regressor'

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1720022127260, experiment_id='1', last_update_time=1720022127260, lifecycle_stage='active', name='nyc-taxi', tags={}>

In [6]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [7]:
import pickle

with open('preprocessor/preprocessor.b', 'rb') as f_in:
    dv = pickle.load(f_in)

In [13]:
X_test = preprocess(df, dv)

In [14]:
target = 'duration'
y_test = df[target].values

In [20]:
%time test_model(name=model_name, stage='Staging', X_test=X_test, y_test=y_test)

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


CPU times: user 10.5 s, sys: 82.7 ms, total: 10.6 s
Wall time: 3.08 s


/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.270842480403794}

In [21]:
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage='Production',
    archive_existing_versions=True,
)

/tmp/ipykernel_42623/1274091239.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1720027548329, current_stage='Production', description='The model version 2 was transtioned to Staging on 2024-07-03 17:32:11.198380', last_updated_timestamp=1720028939454, name='nyc-taxi-regressor', run_id='6c04258530374828bacfc41366c414cc', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/6c04258530374828bacfc41366c414cc/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [22]:
from datetime import datetime

date = datetime.today()

In [23]:
model_version = 2
new_stage = 'production'

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f'The model version {model_version} was transtioned to {new_stage} on {date}'   
)

<ModelVersion: aliases=[], creation_timestamp=1720027548329, current_stage='Production', description=('The model version 2 was transtioned to production on 2024-07-03 '
 '17:51:42.571205'), last_updated_timestamp=1720029105323, name='nyc-taxi-regressor', run_id='6c04258530374828bacfc41366c414cc', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/6c04258530374828bacfc41366c414cc/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>